# Exctraction des fichiers : premières tentatives, en vue d'un script d'automatisation

# Avec PDFQuery : pas pratique

In [1]:
from pdfquery import PDFQuery

In [2]:
# Ouverture du fichier pdf
pdf = PDFQuery("conseil_perfectionnement_A2023_SY02.pdf")
pdf.load()

In [4]:
# Ecriture du fichier xml
pdf.tree.write("conseil_perfectionnement_A2023_SY02.xml", pretty_print=True, encoding="utf-8")

# Avec PyPDF2 et juste SY02

In [5]:
import PyPDF2

In [15]:
# ouverture du fichier pdf
pdfFileObj = open('conseil_perfectionnement_A2023_SY02.pdf', 'rb')
pdfReader = PyPDF2.PdfReader(pdfFileObj)
print("pages : ", len(pdfReader.pages))

pages :  2


In [32]:
for i in range(len(pdfReader.pages)):
    pageObj = pdfReader.pages[i]
    text = pageObj.extract_text()
    text = text.replace("\n ", " ")
    print(text)

Automne 2023  - Rapports du conseil de perfectionnement 
SY02 : Méthodes statistiques pour l'ingénieur                     
Rapport du responsable de l'enseignement : M. Jean-Benoist LEGER
Aucun rapport saisi par le responsable de l'enseignement ce semestre. 
Rapports des conseils de perfectionnement
Aucun rapport des conseils de perfectionnement enregistré.
205 inscrits, 5 absent(s), 80 étudiants reçus à l'uv (Réussite : 40%) - 163 évaluations (81.5%)
- -
-
+
+ +
Q 1Q 2Q 3Q 4Q 5Q 6Q 7Q 8Q 9Q 10
Questions020406080100120140160Question 1
+ + 65.64% 107
+ 23.31% 38
- 5.52% 9
- - 5.52% 9Question 2
+ + 68.1% 111
+ 22.7% 37
- 5.52% 9
- - 3.68% 6Question 3
+ + 26.99% 44
+ 23.93% 39
- 28.22% 46
- - 20.86% 34Question 4
+ + 31.29% 51
+ 31.9% 52
- 26.38% 43
- - 10.43% 17Question 5
+ + 52.15% 85
+ 32.52% 53
- 7.36% 12
- - 7.98% 13
Question 6
+ + 55.83% 91
+ 26.99% 44
- 13.5% 22
- - 3.68% 6Question 7
+ + 76.07% 124
+ 15.95% 26
- 4.29% 7
- - 3.68% 6Question 8
+ + 25.77% 42
+ 17.18% 28
- 26.38% 43
- 

# Avec PyPDF2 et toutes les UVs de A23

In [2]:
import PyPDF2
import re
import json

In [211]:
# ouverture du fichier pdf
pdfFileObj = open('conseil_perfectionnement_complet_A2023.pdf', 'rb')
pdfReader = PyPDF2.PdfReader(pdfFileObj)
print("pages : ", len(pdfReader.pages))

pages :  396


## Analyse d'une seule page

In [252]:
# récupération du texte d'une page
pageObj = pdfReader.pages[2]
text = pageObj.extract_text()
text = text.replace("\n ", " ")

# analyse des données la page
uv_data = {}

# code de l'uv
uv_full_title = re.search("Automne 2023(.*)Rapport du responsable", text, re.DOTALL)
if uv_full_title:
    uv_full_title = uv_full_title.group(1).strip()
    uv_data["code"] = uv_full_title.split(":", 1)[0].strip()
    uv_data["title"] = uv_full_title.split(":", 1)[1].strip().replace("\n", "")

# prof
prof = re.search("Rapport du responsable de l'enseignement : (.*)\n", text)
uv_data["prof"] = prof.group(1).strip() if prof else None

# nb inscrits, taux de réussite
stats = re.search("(\d+) inscrits, (\d+) absent\(s\), (\d+) étudiants reçus à l'uv \(Réussite : (.*)%\) ", text)
if stats:
    uv_data["nb_inscrits"] = int(stats.group(1))
    uv_data["nb_absents"] = int(stats.group(2))
    uv_data["nb_recus"] = int(stats.group(3))
    uv_data["taux_reussite"] = float(stats.group(4))


questions = {}
questions_raw = re.search("Question 1.*Page", text, re.DOTALL)
questions_raw = questions_raw.group().split("Question")
for i in range(1,len(questions_raw)):
    q_notes = re.findall(" (\d+.*)%", questions_raw[i])
    q_notes = [float(i) for i in q_notes]
    questions[i] = {"++": q_notes[0], "+": q_notes[1], "-": q_notes[2], "--": q_notes[3]}
uv_data["questions"] = questions

uv_data

{'code': 'AC02',
 'title': 'Bases de la programmation (apprentissage)',
 'prof': 'M. Ahmed LOUNIS',
 'nb_inscrits': 26,
 'nb_absents': 0,
 'nb_recus': 23,
 'taux_reussite': 88.46,
 'questions': {1: {'++': 57.89, '+': 21.05, '-': 21.05, '--': 0.0},
  2: {'++': 47.37, '+': 42.11, '-': 10.53, '--': 0.0},
  3: {'++': 57.89, '+': 31.58, '-': 10.53, '--': 0.0},
  4: {'++': 21.05, '+': 63.16, '-': 10.53, '--': 5.26},
  5: {'++': 47.37, '+': 42.11, '-': 10.53, '--': 0.0},
  6: {'++': 47.37, '+': 31.58, '-': 15.79, '--': 5.26},
  7: {'++': 31.58, '+': 31.58, '-': 31.58, '--': 5.26},
  8: {'++': 31.58, '+': 15.79, '-': 47.37, '--': 5.26},
  9: {'++': 26.32, '+': 52.63, '-': 15.79, '--': 5.26},
  10: {'++': 26.32, '+': 63.16, '-': 5.26, '--': 5.26}}}

## Analyse de toutes les pages

In [256]:
# données de toutes des uvs du semestre
all_uvs_data = {}

# on parcourt l'ensemble des pages
for i in range(len(pdfReader.pages)):

    # affiche une barre de progression
    print("\r", i, "/", len(pdfReader.pages), end="")

    pageObj = pdfReader.pages[i]
    text = pageObj.extract_text()
    text = text.replace("\n ", " ")

    # si la page ne commence pas par "Automne 2023", on passe à la suivante
    if not text.startswith("Automne 2023"):
        continue
    
    uv_data = {}

    # code de l'uv
    uv_full_title = re.search("Automne 2023(.*)Rapport du responsable", text, re.DOTALL)
    if uv_full_title:
        uv_full_title = uv_full_title.group(1).strip()
        uv_data["code"] = uv_full_title.split(":", 1)[0].strip()
        uv_data["title"] = uv_full_title.split(":", 1)[1].strip().replace("\n", "")

    # prof
    prof = re.search("Rapport du responsable de l'enseignement : (.*)\n", text)
    uv_data["prof"] = prof.group(1).strip() if prof else None

    # nb inscrits, taux de réussite
    stats = re.search("(\d+) inscrits, (\d+) absent\(s\), (\d+) étudiants reçus à l'uv \(Réussite : (.*)%\) ", text)
    if stats:
        uv_data["nb_inscrits"] = int(stats.group(1))
        uv_data["nb_absents"] = int(stats.group(2))
        uv_data["nb_recus"] = int(stats.group(3))
        uv_data["taux_reussite"] = float(stats.group(4))


    questions = {}
    questions_raw = re.search("Question 1.*Page", text, re.DOTALL)
    questions_raw = questions_raw.group().split("Question")
    for i in range(1,len(questions_raw)):
        q_notes = re.findall(" (\d+.*)%", questions_raw[i])
        q_notes = [float(i) for i in q_notes]
        questions[i] = {"++": q_notes[0], "+": q_notes[1], "-": q_notes[2], "--": q_notes[3]}
    uv_data["questions"] = questions

    all_uvs_data[uv_data["code"]] = uv_data 



 395 / 396

In [257]:
# store all_uvs_data in a json file
with open("output_A23.json", "w") as f:
    json.dump(all_uvs_data, f, indent=4)

In [3]:
# get all_uvs_data from a json file
with open("output_A23.json", "r") as f:
    all_uvs_data = json.load(f)

In [4]:
# classe les UVs par taux de réussite (du plus petit au plus grand)
cours_tries = sorted(all_uvs_data.items(), key=lambda x: x[1].get('taux_reussite', 0), reverse=False)

# supprime les UVs dont le taux de réussite n'a pas été renseigné
cours_tries = [i for i in cours_tries if i[1].get('taux_reussite')]

# affiche les UVs par taux de réussite, format : "SY02 : 100% (100 inscrits)".
for i in cours_tries:
    print(f"{i[0]} : {i[1]['taux_reussite']}% ({i[1]['nb_inscrits']} inscrits)")


SY02 : 40.0% (205 inscrits)
MTX2 : 52.63% (21 inscrits)
MQ01 : 54.13% (112 inscrits)
SV01 : 54.29% (72 inscrits)
AI22 : 58.33% (13 inscrits)
MT22 : 60.42% (98 inscrits)
MT23 : 62.06% (342 inscrits)
BA10 : 64.29% (14 inscrits)
AI01 : 64.52% (31 inscrits)
CM13 : 64.86% (38 inscrits)
AM22 : 66.67% (9 inscrits)
BC01 : 66.67% (9 inscrits)
ISC2 : 66.67% (18 inscrits)
UB09 : 66.67% (53 inscrits)
MT09 : 67.8% (61 inscrits)
AM03 : 68.09% (47 inscrits)
SY01 : 68.42% (39 inscrits)
UR02 : 69.44% (72 inscrits)
PS22 : 70.34% (119 inscrits)
PS21 : 70.83% (171 inscrits)
MT33 : 72.0% (25 inscrits)
INF1 : 72.05% (167 inscrits)
AM04 : 72.5% (40 inscrits)
NF04 : 72.5% (41 inscrits)
AI23 : 73.08% (26 inscrits)
MT02 : 73.1% (349 inscrits)
MT03 : 73.25% (425 inscrits)
ISC5 : 73.33% (18 inscrits)
AC01 : 75.0% (45 inscrits)
AI02 : 75.0% (24 inscrits)
AI37 : 75.0% (5 inscrits)
AOS6 : 75.0% (20 inscrits)
LA91 : 75.0% (14 inscrits)
MQ03 : 75.32% (156 inscrits)
UR06 : 75.61% (42 inscrits)
BL10 : 75.68% (37 inscrit